In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from fnope.simulators.ice_simulator.ice_plots import plot_posterior_nice
from fnope.simulators.ice_simulator.modelling_utils import regrid
import torch
import pandas as pd
from fnope.utils.misc import get_data_dir,get_output_dir,read_pickle

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = get_data_dir()
out_dir = get_output_dir()


real_layer_idx = 1
config_nx_iso = 500


# Mimicking how we load the data in the sbi_ice project

setup_df = pd.read_csv(Path(data_dir / "sbi_ice" / "setup_file.csv"))
x_coords = setup_df["x_coord"].to_numpy()
surface = setup_df["surface"].to_numpy()
tmb = setup_df["tmb"].to_numpy()
base = setup_df["base"].to_numpy()

print("x_coords", x_coords.shape)
print("surface", surface.shape)
layer_bounds = read_pickle(Path(data_dir / "sbi_ice" / "layer_bounds.pkl"))
sim_grid = np.linspace(x_coords[0], x_coords[-1], config_nx_iso)
torch_sim_grid = torch.from_numpy(sim_grid).to(device).to(torch.float32)


masks = [sim_grid > bound for bound in layer_bounds]
masks = np.array(masks)
mask = masks[1]

layer_mask = torch.from_numpy(mask)
layer_sparsity = 1
smb_sparsity = 1
layer_mask_slice = torch.zeros(config_nx_iso, dtype=bool)  # pyright: ignore
layer_mask_slice[::layer_sparsity] = 1

layer_all_mask = layer_mask * layer_mask_slice

# Load real layer
layers_df = pd.read_csv(Path(data_dir / "sbi_ice" / "real_layers.csv"))
n_real_layers = len(layers_df.columns) - 2
real_layers = np.zeros(shape=(n_real_layers, 500))
# Regrid the real layers to the simulation grid (e.g. real data is defined on a different grid)
for i in range(n_real_layers):
    layer_depths = regrid(
        layers_df["x_coord"],
        layers_df["layer {}".format(i + 1)],
        sim_grid,
        kind="linear",
    )
    real_layers[i, :] = surface - layer_depths
true_layer_unmasked = torch.tensor(real_layers[1]).float()
true_layer = torch.tensor(real_layers[real_layer_idx][layer_all_mask]).float()


smb_mask_slice = torch.zeros(config_nx_iso, dtype=bool)  # pyright: ignore
smb_mask_slice[::smb_sparsity] = 1
smb_mask = smb_mask_slice  # infer all smb values
smb_x = sim_grid[smb_mask]
layer_x = sim_grid[layer_all_mask]


print(layer_all_mask)

In [ ]:

out_dir = get_output_dir()


predictive_summary_path = out_dir / "ice_experiment/FNOPE_always_equispaced_False/num_sim_100_run_1/real_layers_predictive_simulations_summary.pkl"

predictive_summary = read_pickle(predictive_summary_path)
print(predictive_summary.keys())

In [ ]:
predictive_layers = predictive_summary["best_layers"]
predictive_bmbs = predictive_summary["bmbs"][:, :, 0]
print(predictive_summary["bmbs"].shape)
print(tmb.shape)
predictive_smbs = -predictive_bmbs + torch.from_numpy(tmb).unsqueeze(0)

In [ ]:
print(true_layer.shape)

In [ ]:
with plt.rc_context(fname="matplotlibrc"):

    plot_posterior_nice(
        x=x_coords,
        mb_mask=smb_mask,
        tmb=tmb,
        posterior_smb_samples=predictive_smbs,
        LMI_boundary=x_coords[layer_all_mask][0],
        layer_mask=layer_all_mask,
        posterior_layer_samples=predictive_layers.cpu().numpy()[:, layer_all_mask],
        posterior_layer_ages=predictive_summary["ages"],
        true_layer=true_layer_unmasked.cpu().numpy(),
        shelf_base=base,
        shelf_surface=surface,
        plot_only_predictive=True,
        figsize=(3.5, 1.3),
    )

    # plt.savefig(
    #     "ice_plots/post_predictives.svg",
    #     bbox_inches="tight",
    #     dpi=300,
    # )